In [234]:
from helper_loader import *
from histalign.backend.maths import apply_rotation, compute_normal

from histalign.backend.models import VolumeSettings

from math import sin, cos

In [235]:
settings = VolumeSettings(
    orientation=Orientation.CORONAL,
    resolution=Resolution.MICRONS_50,
    offset=100,
    pitch=90,
    yaw=0,
    shape=(264, 160, 228),
)

In [236]:
center = np.array(
    [
        (settings.shape[0] - 1) // 2 + 1,
        (settings.shape[1] - 1) // 2,
        (settings.shape[2] - 1) // 2,
    ]
)
center

array([132,  79, 113])

In [237]:
image_center = center.copy()
image_center[0] -= settings.offset
image_center

array([ 32,  79, 113])

In [238]:
first_half = (center[1] + 1) / cos(settings.pitch)
first_half

-178.54209022862372

In [239]:
x_axis = vedo.Arrow(end_pt=(300, 0, 0), c="red", s=0.5)
y_axis = vedo.Arrow(end_pt=(0, 200, 0), c="yellow", s=0.5)
z_axis = vedo.Arrow(end_pt=(0, 0, 250), c="blue", s=0.5)

axes = [x_axis, y_axis, z_axis]

In [240]:
custom_coronal_camera = coronal_camera.copy()
custom_coronal_camera["position"] = center.copy()
custom_coronal_camera["position"][0] += 800
custom_coronal_camera["focal_point"] = center

In [241]:
box = vedo.Box(
    pos=(0, settings.shape[0], 0, settings.shape[1], 0, settings.shape[2]),
)

horizontal_plane_normal = np.array([0, -1, 0])
rotated_horizontal_plane_normal = apply_rotation(horizontal_plane_normal, settings)

show(
    [
        box,
        *axes,
    ],
    camera=custom_coronal_camera,
)

In [242]:
show(
    [
        vedo.Arrow(end_pt=compute_normal(settings) * 200, s=1),
        vedo.Arrow(end_pt=rotated_horizontal_plane_normal * 200, s=1),
        *axes,
    ],
    camera=custom_coronal_camera,
)

In [243]:
point1 = image_center - rotated_horizontal_plane_normal * 1000
point2 = image_center + rotated_horizontal_plane_normal * 1000

intersections = box.intersect_with_line(point1, point2)

In [244]:
rotated_horizontal_plane_normal

array([ 1.000000e+00, -6.123234e-17,  0.000000e+00])

In [245]:
point1

array([-968.,   79.,  113.])

In [246]:
point2

array([1032.,   79.,  113.])

In [247]:
show(
    [
        box.alpha(0.2),
        vedo.Point(center, r=20, c="red"),
        vedo.Point(image_center, r=20, c="green"),
        vedo.Line(point1, point2, c="purple", lw=10),
        vedo.Points(box.intersect_with_line(point1, point2), r=20, c="blue"),
    ],
    camera=custom_coronal_camera,
)

In [248]:
from scipy.spatial import distance

point1_to_image_center = distance.euclidean(intersections[0], image_center)
print(point1_to_image_center)
point2_to_image_center = distance.euclidean(intersections[1], image_center)
print(point2_to_image_center)

32.0
232.0
